In [1]:
!pip install transformers accelerate gradio kaggle -q
!pip install -U transformers -q
!pip install -U datasets

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.0 which is incompatible.
dask-cudf 23.8.0 requires dask==2023.7.1, b

In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
# Add the EOS token as the padding token
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer.special_tokens_map)

{'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}


In [ ]:
hub_model_name = "JokesGPT"
model.push_to_hub(hub_model_name, use_auth_token="enter_your_token_here")
tokenizer.push_to_hub(hub_model_name, use_auth_token="enter_your_token_here")

In [6]:
from datasets import load_dataset

dataset = load_dataset("Amirkid/jokes")

Generating train split:   0%|          | 0/578634 [00:00<?, ? examples/s]

In [7]:
def preprocess(ex):
    ex["text"] = ["JOKE: " + x for x in ex["text"] if x]
    ex = tokenizer(ex["text"], truncation=True)
    return ex
tokenized_dataset = dataset.map(preprocess, batched=True, batch_size = 2000, num_proc=8, remove_columns=dataset["train"].features)

Map (num_proc=8):   0%|          | 0/578634 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer)

In [8]:
# loader = DataLoader(tokenized_dataset["train"], collate_fn = data_collator, batch_size = 8)
# for a in loader:
#     print(a["input_ids"])
#     break

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
EPOCHS = 1
BATCH_SIZE = 4
loader = DataLoader(tokenized_dataset["train"], collate_fn = data_collator, batch_size = BATCH_SIZE)
dataset_size = tokenized_dataset["train"].num_rows 
num_batches = dataset_size/BATCH_SIZE
num_training_steps = EPOCHS * num_batches

progress_bar = tqdm(total = num_training_steps)

lr = 5e-5
optimizer = torch.optim.AdamW(model.parameters(), lr = lr)
device = "cuda" if torch.cuda.is_available() else "cpu"

losses = []
model.to(device)
model.train()

actual_idx = 0
for a in loader:
    if actual_idx % 100 == 0 and actual_idx != 0:
        print(f"saving model for iteration: {actual_idx}")
        print(f"Loss: {sum(losses)/len(losses)}")
        print(f"-----------------------")
        model.push_to_hub(hub_model_name, commit_message = f"iteration {actual_idx}", use_auth_token="enter_your_token_here")
        
    if actual_idx % 100 == 0 and actual_idx != 0:
        print(f"Loss: {sum(losses)/len(losses)}")
    
    actual_idx += 1
    input_ids, attention_mask = a["input_ids"], a["attention_mask"]
    input_ids, attention_mask = input_ids.type(torch.long).to(device), attention_mask.type(torch.long).to(device)
    out = model(input_ids = input_ids,
                attention_mask=attention_mask,
               labels = input_ids)
    
    loss, logits = out[:2]
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())

#     progress_bar.update(1)

  0%|          | 0/143539.75 [00:00<?, ?it/s]

saving model for iteration: 100
Loss: 2.09256292283535
-----------------------


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Loss: 2.09256292283535
saving model for iteration: 200
Loss: 2.0388212782144546
-----------------------


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Loss: 1.970009876526892
saving model for iteration: 1500
Loss: 1.9627266168743371
-----------------------


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Loss: 1.9627266168743371
saving model for iteration: 1600
Loss: 1.9551352923503145
-----------------------


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]